In [1]:
!pip install albumentations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import albumentations as A
import cv2
import os
import json
import math
from glob import glob

# augmentation pipeline using Albumentations library
augmentation_pipeline = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.7),
    A.RandomScale(scale_limit=0.2, p=0.5),
    A.PadIfNeeded(min_height=256, min_width=256, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.HorizontalFlip(p=0.5)
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))

input_folder = "/content/drive/MyDrive/Exercise Tracking/Annotated Exercise Frames"  # original anotated images 
output_folder = "/content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames"  # augmented images

if not os.path.exists(output_folder): # create output folder if not exist
    os.makedirs(output_folder)

for exercise_folder in glob(f"{input_folder}/*"): # iterate over all exercise folders
    exercise_name = os.path.basename(exercise_folder)

    for subfolder in glob(f"{exercise_folder}/*"):
        subfolder_name = os.path.basename(subfolder)
        output_subfolder = os.path.join(output_folder, exercise_name, subfolder_name)

        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)

        for image_path in glob(f"{subfolder}/*.jpg"): # read images & keypoints from JSON files
            image = cv2.imread(image_path)
            json_path = image_path.replace(".jpg", ".json")

            if not os.path.exists(json_path):
                print(f"Warning: JSON file for {image_path} not found.")
                continue

            with open(json_path, "r") as f:
                data = json.load(f)
                keypoints = data.get("keypoints", [])

            valid_keypoints = [] # filter out invalid keypoints
            for kp in keypoints:
                if kp['x'] is not None and kp['y'] is not None and not (math.isnan(kp['x']) or math.isnan(kp['y'])):
                    valid_keypoints.append((kp['x'], kp['y']))

            augmented = augmentation_pipeline(image=image, keypoints=valid_keypoints) # apply augmentation
            augmented_image = augmented["image"] # get augmented image
            augmented_keypoints_xy = augmented["keypoints"] # get augmented keypoints

            augmented_keypoints = [ # update keypoints with augmented values
                {'part': kp['part'], 'x': augmented_kp[0], 'y': augmented_kp[1]}
                for kp, augmented_kp in zip(keypoints, augmented_keypoints_xy)
                if kp['x'] is not None and kp['y'] is not None and not (math.isnan(kp['x']) or math.isnan(kp['y']))
            ]

            image_name = os.path.basename(image_path)
            augmented_image_path = os.path.join(output_subfolder, f"aug_{image_name}")

            save_image_result = cv2.imwrite(augmented_image_path, augmented_image) # save augmented image
            if save_image_result:
                print(f"Saved augmented image: {augmented_image_path}")
            else:
                print(f"Failed to save augmented image: {augmented_image_path}")

            augmented_json_path = augmented_image_path.replace(".jpg", ".json") # save augmented keypoints in JSON file
            augmented_data = {"keypoints": augmented_keypoints}

            try:
                with open(augmented_json_path, "w") as f:
                    json.dump(augmented_data, f)
                print(f"Saved augmented JSON: {augmented_json_path}")
            except Exception as e:
                print(f"Failed to save JSON file: {augmented_json_path} with error: {e}")

print("Augmentation completed, and all images and JSON files are saved.")


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Saved augmented image: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_0.jpg
Saved augmented JSON: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_0.json
Saved augmented image: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_1.jpg
Saved augmented JSON: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_1.json
Saved augmented image: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_2.jpg
Saved augmented JSON: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_2.json
Saved augmented image: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Turns/Head Turn 1/aug_frame_3.jpg
Saved augmented JSON: /content/drive/MyDrive/Exercise Tracking/Augmented Exercise Frames/Head Tur